In [78]:
import googlemaps
# from datetime import datetime
import pandas as pd

In [81]:
gmaps = googlemaps.Client(key='GOOGLE_MAP_API_KEY')

# Geocoding an address
geocode_result = gmaps.geocode('9450 Gilman Drive, La Jolla, CA')

# # Look up an address with reverse geocoding
# reverse_geocode_result = gmaps.reverse_geocode((40.714224, -73.961452))

# # Request directions via public transit
# now = datetime.now()
# directions_result = gmaps.directions("Sydney Town Hall",
#                                      "Parramatta, NSW",
#                                      mode="transit",
#                                      departure_time=now)

# # Validate an address with address validation
# addressvalidation_result =  gmaps.addressvalidation(['1600 Amphitheatre Pk'], 
#                                                     regionCode='US',
#                                                     locality='Mountain View', 
#                                                     enableUspsCass=True)

In [9]:
# geocode_result

In [10]:
# reverse_geocode_result = gmaps.reverse_geocode((32.892903, -117.262129))
# reverse_geocode_result

In [79]:
stores = pd.read_csv('data/store_dim.csv')
stores.head()

,store_key,division,district,upazila
0,S0001,SYLHET,HABIGANJ,AJMIRIGANJ
1,S0002,SYLHET,HABIGANJ,BAHUBAL
2,S0003,SYLHET,HABIGANJ,BANIACHONG
3,S0004,SYLHET,HABIGANJ,CHUNARUGHAT
4,S0005,SYLHET,HABIGANJ,HABIGANJ SADAR


## district

In [37]:
stores['district'].unique()

array(['HABIGANJ', 'MAULVIBAZAR', 'SUNAMGANJ', 'SYLHET', 'DINAJPUR',
       'GAIBANDHA', 'KURIGRAM', 'LALMONIRHAT', 'NILPHAMARI', 'PANCHAGARH',
       'RANGPUR', 'THAKURGAON', 'BOGRA', 'JOYPURHAT', 'NAOGAON', 'NATORE',
       'CHAPAI NABABGANJ', 'PABNA', 'RAJSHAHI', 'SIRAJGANJ', 'BAGERHAT',
       'CHUADANGA', 'JESSORE', 'JHENAIDAH', 'KHULNA', 'KUSHTIA', 'MAGURA',
       'MEHERPUR', 'NARAIL', 'SATKHIRA', 'DHAKA', 'FARIDPUR', 'GAZIPUR',
       'GOPALGANJ', 'JAMALPUR', 'KISHOREGONJ', 'MADARIPUR', 'MANIKGANJ',
       'MUNSHIGANJ', 'MYMENSINGH', 'NARAYANGANJ', 'NARSINGDI',
       'NETRAKONA', 'RAJBARI', 'SHARIATPUR', 'SHERPUR', 'TANGAIL',
       'BANDARBAN', 'BRAHMANBARIA', 'CHANDPUR', 'CHITTAGONG', 'COMILLA',
       "COX'S BAZAR", 'FENI', 'KHAGRACHHARI', 'LAKSHMIPUR', 'NOAKHALI',
       'RANGAMATI', 'BARGUNA', 'BARISAL', 'BHOLA', 'JHALOKATI',
       'PATUAKHALI', 'PIROJPUR'], dtype=object)

In [73]:
district_coordinates = {}
for district in stores['district'].unique():
    if district  == 'MAGURA': 
        district = 'MAGURA, KHULNA'
    if district  == 'GOPALGANJ':
        district = 'GOPALGANJ, DHAKA'
    output = gmaps.geocode(district)[0]['geometry']['location']
    # print(district,output)
    district_coordinates[district] = output

In [75]:
# district_coordinates

In [76]:
df = pd.DataFrame(district_coordinates)
df

,HABIGANJ,MAULVIBAZAR,SUNAMGANJ,SYLHET,DINAJPUR,GAIBANDHA,KURIGRAM,LALMONIRHAT,NILPHAMARI,PANCHAGARH,...,KHAGRACHHARI,LAKSHMIPUR,NOAKHALI,RANGAMATI,BARGUNA,BARISAL,BHOLA,JHALOKATI,PATUAKHALI,PIROJPUR
lat,24.389620,24.485853,25.066655,24.904781,25.622148,25.328978,25.813307,25.916198,25.936273,26.335376,...,23.113711,22.945403,22.865294,22.661956,22.160505,22.713288,22.178531,22.641138,22.358924,22.584105
lng,91.412622,91.776460,91.407239,91.860008,88.643796,89.541507,89.648288,89.450580,88.840704,88.551698,...,91.979500,90.834162,91.097059,92.161112,90.120064,90.349628,90.710102,90.198700,90.328013,89.972013


In [77]:
df.to_csv('district_coordinates.csv', index=False)

## division

In [80]:
stores['division'].unique()

array(['SYLHET', 'RANGPUR', 'RAJSHAHI', 'KHULNA', 'DHAKA', 'CHITTAGONG',
       'BARISAL'], dtype=object)

In [91]:
divisions_coordinates = {}
for division in stores['division'].unique():
    output = gmaps.geocode(division)[0]['geometry']['location']
    # print(division,output)
    divisions_coordinates[division] = output

In [85]:
divisions_coordinates

{'SYLHET': {'lat': 24.9047808, 'lng': 91.86000790000001},
 'RANGPUR': {'lat': 25.7493938, 'lng': 89.26108649999999},
 'RAJSHAHI': {'lat': 24.3746497, 'lng': 88.60036649999999},
 'KHULNA': {'lat': 22.8373287, 'lng': 89.54004719999999},
 'DHAKA': {'lat': 23.804093, 'lng': 90.4152376},
 'CHITTAGONG': {'lat': 22.3752075, 'lng': 91.8348606},
 'BARISAL': {'lat': 22.7132876, 'lng': 90.3496278}}

In [87]:
df = pd.DataFrame(divisions_coordinates)
df

,SYLHET,RANGPUR,RAJSHAHI,KHULNA,DHAKA,CHITTAGONG,BARISAL
lat,24.904781,25.749394,24.374650,22.837329,23.804093,22.375207,22.713288
lng,91.860008,89.261086,88.600366,89.540047,90.415238,91.834861,90.349628


In [88]:
df.to_csv('division_coordinates.csv', index=False)

## upazila

In [117]:
unique_upazila = stores[['division','district', 'upazila']].drop_duplicates(keep='last')
unique_upazila['upazila_str'] = stores['upazila'] + ', ' + stores['district'] +  ', ' + stores['division']
unique_upazila

,division,district,upazila,upazila_str
0,SYLHET,HABIGANJ,AJMIRIGANJ,"AJMIRIGANJ, HABIGANJ, SYLHET"
1,SYLHET,HABIGANJ,BAHUBAL,"BAHUBAL, HABIGANJ, SYLHET"
2,SYLHET,HABIGANJ,BANIACHONG,"BANIACHONG, HABIGANJ, SYLHET"
3,SYLHET,HABIGANJ,CHUNARUGHAT,"CHUNARUGHAT, HABIGANJ, SYLHET"
4,SYLHET,HABIGANJ,HABIGANJ SADAR,"HABIGANJ SADAR, HABIGANJ, SYLHET"
...,...,...,...,...
721,BARISAL,BARISAL,HIZLA,"HIZLA, BARISAL, BARISAL"
722,BARISAL,BARISAL,BARISAL SADAR (KOTWALI),"BARISAL SADAR (KOTWALI), BARISAL, BARISAL"
723,BARISAL,BARISAL,MEHENDIGANJ,"MEHENDIGANJ, BARISAL, BARISAL"
724,BARISAL,BARISAL,MULADI,"MULADI, BARISAL, BARISAL"


In [131]:
upazilas_coordinates = {}
for upazila in unique_upazila['upazila_str'].unique():
    output = gmaps.geocode(upazila)[0]['geometry']['location']
    # print(upazila,output)
    upazila = upazila.split(',')[0]
    upazilas_coordinates[upazila] = output

In [133]:
# upazilas_coordinates

In [134]:
df = pd.DataFrame(upazilas_coordinates)
df

,AJMIRIGANJ,BAHUBAL,BANIACHONG,CHUNARUGHAT,HABIGANJ SADAR,LAKHAI,MADHABPUR,NABIGANJ,BARLEKHA,JURI,...,AGAILJHARA,BABUGANJ,BAKERGANJ,BANARI PARA,GAURNADI,HIZLA,BARISAL SADAR (KOTWALI),MEHENDIGANJ,MULADI,WAZIRPUR
lat,24.548922,24.341248,24.506322,24.197358,24.384100,24.310610,24.099822,24.567188,24.707272,24.597096,...,22.971329,22.713288,22.713288,22.701002,22.974237,22.911019,22.713288,22.826070,22.713288,22.819293
lng,91.236171,91.537811,91.356569,91.517232,91.416879,91.302368,91.296327,91.513601,92.191702,92.114342,...,90.163544,90.349628,90.349628,90.353451,90.222121,90.511607,90.349628,90.526964,90.349628,90.250270


In [135]:
df.dropna()

,AJMIRIGANJ,BAHUBAL,BANIACHONG,CHUNARUGHAT,HABIGANJ SADAR,LAKHAI,MADHABPUR,NABIGANJ,BARLEKHA,JURI,...,AGAILJHARA,BABUGANJ,BAKERGANJ,BANARI PARA,GAURNADI,HIZLA,BARISAL SADAR (KOTWALI),MEHENDIGANJ,MULADI,WAZIRPUR
lat,24.548922,24.341248,24.506322,24.197358,24.384100,24.310610,24.099822,24.567188,24.707272,24.597096,...,22.971329,22.713288,22.713288,22.701002,22.974237,22.911019,22.713288,22.826070,22.713288,22.819293
lng,91.236171,91.537811,91.356569,91.517232,91.416879,91.302368,91.296327,91.513601,92.191702,92.114342,...,90.163544,90.349628,90.349628,90.353451,90.222121,90.511607,90.349628,90.526964,90.349628,90.250270


In [137]:
df.to_csv('upazila_coordinates.csv', index=False)